# Results

## General

We're considering performace of an SVM classifier (OVR scheme) for 4 classes, using three different kernel functions. To train and test the SVM classifier we'll have to provide it with the similarity matrices.

Training - requires NxN Gram matrix (square matrix compromised of values of the kernel function between pairs training examples)

Testing - requires MxN matrix (element i,j is the value of the kernel function between i-th example of the testing set and j-th example of the training set)

N = cardinaliry of the training set, M = cardinality of the testing set

In [10]:
import pandas as pd
import numpy as np

In [3]:
def similarity_matrix(first_dataset, second_dataset, similarity_function):
    """
    Calculate the similarity matrix between elements of two datasets using a similarity function.

    Args:
    - first_dataset: List or array-like, the first dataset
    - second_dataset: List or array-like, the second dataset
    - similarity_function: Function, the similarity function that takes two elements as arguments

    Returns:
    - similarity_matrix: NumPy ndarray, the similarity matrix
    """

    # Ensure that the datasets are iterable
    first_dataset = np.array(first_dataset)
    second_dataset = np.array(second_dataset)

    # Initialize an empty similarity matrix
    similarity_matrix = np.zeros((len(first_dataset), len(second_dataset)))

    # Populate the similarity matrix
    for i in range(len(first_dataset)):
        for j in range(len(second_dataset)):
            similarity_matrix[i, j] = similarity_function(first_dataset[i], second_dataset[j])

    return similarity_matrix

### Kernels

We're considering SSK, NGK and WK kernel functions, each with its own set of hiperparameters. 

SSK is parameterised by $k$ = length of the substrings used for feature mapping/kernel computation and $\lambda$= real numer from the interval [0,1] which indicates how much we penalise the noncontiguity of the appeared substring in the imput document. 

NGK is parameterised with $n$, corresponding to $k$ in SSk.

WK has no hiperparameters

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support


def WK_SWM(X_train, y_train, X_test, y_test):  #a multi-class classifier

    '''calculates f1, precision, and recall for a SVM classifer using linear tfidf mapping
    Args:
    - X_train,y_train, X_test, y_test
    Returns:
    - f1, precision, recall: for each of the classes in form of a pandas dataframe w columns Kernel, Class, F1, Precision , Recall
    '''
    vectorizer = TfidfVectorizer(sublinear_tf=True, use_idf=True, smooth_idf=True, norm='l2',
                                 analyzer='word', stop_words='english')

    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    svm_classifier = SVC(kernel='linear')
    svm_classifier.fit(X_train_tfidf, y_train)

    y_pred = svm_classifier.predict(X_test_tfidf)

    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
    results_df = pd.DataFrame({
    'Kernel' : 'WK',
    'Class': range(len(precision)),
    'F1-Score': f1,
    'Precision': precision,
    'Recall': recall,
    })
    return results_df


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

def NGK_SVM(X_train, y_train, X_test, y_test, k):
    '''calculates f1, precision, and recall for a SVM classifer using linear n-gram mapping
    Args:
    - X_train,y_train, X_test, y_test, k=lenthg of the ngrams
    Returns:
    - f1, precision, recall: for each of the classes in form of a pandas dataframe w columns Kernel,k, Class, F1, Precision , Recall
    '''
    ngram_range = (k, k)
    vectorizer = CountVectorizer(analyzer='char', ngram_range=ngram_range)

    x_train_ngrams = normalize(vectorizer.fit_transform(X_train), norm='l2')  #ngram vectors normalised to l2 norm
    x_test_ngrams = normalize(vectorizer.transform(X_test), norm='l2')

    svm_classifier = SVC(kernel='linear')
    svm_classifier.fit(x_train_ngrams, y_train)

    y_pred = svm_classifier.predict(x_test_ngrams)

    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)

    results_df = pd.DataFrame({
        'Kernel':'NGK',
        'k':k,
        'Class': range(len(precision)),
        'F1-Score': f1,
        'Precision': precision,
        'Recall': recall
        })
    return results_df

In [1]:
from main import sskpy
def ssk(a, b, k, lambd):
    return sskpy(a,b,k,lambd)

In [2]:
def ssk_partial(ka, Lambda):
    return lambda a,b: ssk(a,b,k=ka, lambd=Lambda)

In [3]:
def SSK_SVM(X_train, y_train, X_test, y_test, k, lambd ):
    '''calculates f1, precision, and recall for a SVM classifer using SSK
    Args:
    - X_train,y_train, X_test, y_test, k=lenthg of the substrings, lambd= weight decay factor
    Returns:
    - f1, precision, recall: for each of the classes in form of a pandas dataframe w columns Kernel,k,lambd, Class, F1, Precision , Recall
    '''
    # calculate gram matrix for training and matrix for prediction
    kernel_function=ssk_partial(k, lambd)
    train_matrix=similarity_matrix(X_train, X_train, kernel_function)
    test_matrix=similarity_matrix(X_test, X_train), kernel_function

    #model - precomputed, trained on the gram matrix
    svm_model=SVC(kernel='precomputed')
    svm_model.fit(train_matrix, y_train)

    #predicting
    y_pred=svm_model.predict(test_matrix)

    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)

    results_df = pd.DataFrame({
        'Kernel':'SSK',
        'k':k,
        'lambda': lambd,
        'Class': range(len(precision)),
        'F1-Score': f1,
        'Precision': precision,
        'Recall': recall
        })
    return results_df

## Experiments

### 0.Creating datasets
each run has to be repeated 10 times - we need 10 pairs of (train, test) sets that need to satisfy the properties given in the paper:

 earn 152 (40); acq 114 (25); crude 76 (15); grain 38 (10)
 since not enough examples of corn I'll use grain as the 4th topic

In [8]:
data=pd.read_csv('../data/preprocessed.csv')[['topics', 'body']].copy()
topic_mapping = {'earn': 0, 'acq': 1, 'crude': 2, 'grain': 3}
data['topics'] = data['topics'].map(topic_mapping)

class0df=data[data.topics==0].head(200)  #earn
class1df=data[data.topics==1].head(200)  #acq
class2df=data[data.topics==2].head(200)  #crude
class3df=data[data.topics==3].head(200)  #grain

final=pd.concat([class0df, class1df, class2df, class3df])
print(len(final))

800


In [12]:
from sklearn.model_selection import StratifiedKFold

X_all=np.array(final['body']).reshape(len(final),1)
y_all=np.array(final['topics'])

n_folds = 10
stratified_kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
datasets = []

for train_index, test_index in stratified_kfold.split(X_all,y_all):
    X_train, X_test = X_all[train_index], X_all[test_index]
    y_train, y_test = y_all[train_index], y_all[test_index]
    
    datasets.append((X_train, X_test, y_train, y_test))

# dataset[i] = Xtrain_ i , Xtest_i, ytrain_i, ytest_i, i=1,...,10 aka the index of the run

### Experiment 1 - Varying Subsequence Length in SSK and NGK

For this experimet we're keeping lambda for SSK at 0.5, and varying k/n for SSK/NGKin values [3,4,5,6,7,8,10,12,14], WK has no parameters

In [10]:
lambda_default=0.5
kvalues=[3,4,5,6,7,8,10,12,14]

### Experiment 2 - Varying Weight Decay Factor in SSK

### Experiment 3 - Combining two SSK of Different Substring Lengths

### Experiment 4 - Combining SSK and NGK 

length of both kernels =5, lambda for SSK=0.5, varying the contibutions pf SSK and NKG 

### Experiment 5 - Combining two SSK of Different Weight Decay Factors